# GINへの同期の失敗を解消する。

GINへの同期で失敗した場合は以下の各セッションをに順次実行してください。<br>
同期が失敗した場合、実行セルの出力結果に`リポジトリ側の変更と競合しました。競合を解決してください。`と表示されます。<br>
それ以外の時は、以下の処理は実行しないでください。

## 0. コンフリクトが発生しているファイルを特定する。

In [ ]:
import os
from utils import display_util
from utils.common import common
from utils.git import git_module
from IPython.display import clear_output

conflict_filepaths = git_module.get_conflict_filepaths()
annex_path_list = git_module.get_annex_content_file_paht_list()

# get 'git-annex' conflict path
annex_conflict_filepaths = common.get_AND_elements(conflict_filepaths, annex_path_list)
# get 'git' conflict path
git_conflict_filepaths = list(set(conflict_filepaths) - set(annex_conflict_filepaths))
clear_output()

In [ ]:
print(git_conflict_filepaths)
print(annex_conflict_filepaths)

In [ ]:
# get conflict variants
from utils.common import common
stdout, stderr, rt = common.exec_subprocess('git annex resolvemerge', False)
result = stdout.decode('utf-8')
print(result)

from utils import display_util

if len(git_conflict_filepaths) > 0:
    display_util.display_info(msg='『1. 競合ファイル(git)を修正する。』を必ず実行してください。')
if len(annex_conflict_filepaths) > 0:
    display_util.display_info(msg='『2. 競合ファイル(git-annex)を修正する。』を必ず実行してください。')


## 1. 競合ファイル(git)を修正する。

以下のタスクを実行すると、競合が起きているファイルパスが順次、表示されます。<br>
以下の手順に従って競合ファイルを修正してください。<br>

1. 下のセルを実行する。
2. 競合ファイルのパスを確認する。
   
※ 競合が発生したファイルパスと`修正完了`ボタンが表示されます。<br>
※ この時点では、`修正完了`ボタンをクリックしないください。

![競合ファイルパス](./images/Display_of_conflicting_file_paths.png)

3. ダッシュボートの`File` > `Open...`と順にクリックする。
4. 競合が発生しているファイルにチェックを入れ、`edit`ボタンをクリックする。
5. ファイルを編集して競合を解消する。

* 編集前

![編集前](./images/Before_conflict_resolution.png)

* 編集後

![編集前](./images/After_conflict_resolution.png)

1. ダッシュボートの`File` > `Save`と順にクリックし、保存する。
2. `修正完了`ボタンをクリックする。
   1. 複数競合が発生している場合は、手順[2. 競合ファイルのパスを確認する]に戻り、次の競合ファイルの操作を行ってください。
   2. 全ての競合解消操作が完了した場合、`次にお進みください。`とメッセージ表示されます。次にお進みください。


In [ ]:
# ユーザにgit管理ファイルを手動で編集してもらう。
from ipywidgets import Button
from utils import display_util
from IPython.display import display

def on_click_callback(clicked_button: Button) -> None:
    clicked_button.button_style = 'success'
    display_util.display_info(msg="次にお進みください。")

display_util.display_msg(msg='競合が発生しているファイルは以下です。', fore=None, back=None, tag='p')
for i, git_conflict_filepath in enumerate(git_conflict_filepaths):
    display_util.display_msg(msg='{} : {}'.format(int(i) + 1, git_conflict_filepath), fore=None, back=None, tag='p')
    button = Button(description='修正完了', button_style='')
    button.on_click(on_click_callback)
    display(button)


In [ ]:
# add git content
from utils.common import common
import os
os.chdir(os.environ['HOME'])
for git_conflict_filepath in git_conflict_filepaths:
    stdout, stderr, rt = common.exec_subprocess('git add {}'.format(git_conflict_filepath), False)
    result = stdout.decode('utf-8')

## 2. 競合ファイル(git-annex)を修正する。

In [ ]:
from datalad import api
import os
from utils.git import git_module

os.chdir(os.environ['HOME'])
remote_annex_variant_paths =  git_module.get_remote_annex_variant_path(annex_conflict_filepaths)

# get information of datalad
multi_status_info = api.status(path=['experiments/'])

# create git-annex varitants dict
variants_dict = dict[str, list[str]]()
for conflict_annex_path in annex_conflict_filepaths:
    variants_dict[conflict_annex_path] = list[str]()

for unit_status_info in multi_status_info:
    if unit_status_info['state'] == 'added':
        for conflict_annex_path in annex_conflict_filepaths:
            # adjust file path that is conflicted for extract variant paht
            dirpath = os.path.dirname(conflict_annex_path)
            filename_no_extension = os.path.splitext(os.path.basename(conflict_annex_path))[0]
            target_path = '{}/{}.variant-'.format(dirpath, filename_no_extension)

            path = unit_status_info['path']
            refds = '{}/'.format(unit_status_info['refds'])
            path = path.replace(refds, '', 1)
            if path.startswith(target_path):
                variant_list = variants_dict[conflict_annex_path]
                variant_list.append(path)
print(variants_dict)

In [ ]:
from ipywidgets import Button, Dropdown
from utils import display_util
from IPython.display import display

select_action = list()
def on_click_callback(clicked_button: Button) -> None:
    clicked_button.button_style = 'success'
    select = dropdown.value
    select_action.append(select)
    

menu = ['両方残す', 'HEADのファイルを残す', 'Remoteのファイルを残す']
dropdown = Dropdown(
    options=menu,
    description='アクション:',
    disabled=False,
    style= {'description_width': 'initial'}
)

display_util.display_msg(msg='競合が発生しているファイルは以下です。', fore=None, back=None, tag='p')
for i, conflict_filepath in enumerate(variants_dict):
    display_util.display_msg(msg='{} : {}'.format(int(i) + 1, conflict_filepath), fore=None, back=None, tag='p')
    print(variants_dict[conflict_filepath])
    display_util.display_msg(msg='HEAD : {}'.format(variants_dict[conflict_filepath][0]), fore=None, back=None, tag='p')
    display_util.display_msg(msg='Remote : {}'.format(variants_dict[conflict_filepath][1]), fore=None, back=None, tag='p')
    button = Button(description='確定', button_style='')
    button.on_click(on_click_callback)
    display(dropdown, button)

In [ ]:
print(select_action)

action_by_content = dict[str, str]()
for i, conflict_filepath in enumerate(variants_dict):
    action_by_content[conflict_filepath] = select_action[i]





## 3. GINへ修正内容をプッシュする。

In [ ]:
from utils.common import common
import os
os.chdir(os.environ['HOME'])
commit_msg = 'conflict resolution'
stdout, stderr, rt = common.exec_subprocess('git commit -m "{}"'.format(commit_msg), False)
result = stdout.decode('utf-8')

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
from IPython.display import clear_output
import traceback
# Push to GIN
try:
    utils.push()
    clear_output()
except:
    datalad_error = traceback.format_exc()
    display_util.display_err(msg='[ERROR] : '.format(datalad_error))
    display_util.display_err(msg='GIN-forkへのコミットのプッシュが失敗しました。')
else:
    display_util.display_info(msg='GIN-forkへのコミットのプッシュが成功しました。\nこれで競合解消タスクは完了です。')
    display_util.display_warm(msg='先ほどGIN-forkへの同期が失敗したNotebookに戻り、再度、セルを実行してください。')
    